In [15]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import re
import networkx as nx
import pygad

num_epochs = 200
batch_size = 32
learning_rate = 0.001
hidden_dim = 16

# Genetic Algorithm parameters
population_size = 10
num_generations = 10
crossover_probability = 0.8
mutation_probability = 0.2
parent_selection_type = "rank"
crossover_type = "uniform"
crossover_type_param = 2
elitism = True

global model, train_loader

# Edge Feature Class
class edgeFeatures(object):
    def __init__(self, label=None, type=None, embeddings=None):
        self.label = label
        self.type = type
        self.embeddings = embeddings
        return

# Function to handle the graph
def structuralGraph(realFileName, fakeFileName, dataset):
    
    dataReal = pd.read_csv(realFileName, sep=' ', skiprows=0)
    dataFake = pd.read_csv(fakeFileName, sep=' ', skiprows=0)

    train_Real_Graph = nx.Graph()
    train_Fake_Graph = nx.Graph()
    test_Real_Graph = nx.Graph()
    test_Fake_Graph = nx.Graph()

    real_edge_Attritube = np.array(dataReal.iloc[:, 0:3])
    fake_edge_Attritube = np.array(dataFake.iloc[:, 0:3])

    lenReal = len(real_edge_Attritube)
    lenFake = len(fake_edge_Attritube)

    if dataset.lower() == 'facebook':
        dataNewType = [9, 8, 7, 6, 5, 4]
    else:
        dataNewType = [2]

    for i in range(lenReal):
        relation = real_edge_Attritube[i][2]
        if relation in dataNewType:
            test_Real_Graph.add_edge(real_edge_Attritube[i][0], real_edge_Attritube[i][1], relationship=relation)
        else:
            train_Real_Graph.add_edge(real_edge_Attritube[i][0], real_edge_Attritube[i][1], relationship=relation)

    for i in range(lenFake):
        relation = fake_edge_Attritube[i][2]
        if relation in dataNewType:
            test_Fake_Graph.add_edge(fake_edge_Attritube[i][0], fake_edge_Attritube[i][1], relationship=relation)
        else:
            train_Fake_Graph.add_edge(fake_edge_Attritube[i][0], fake_edge_Attritube[i][1], relationship=relation)

    return train_Real_Graph, train_Fake_Graph, test_Real_Graph, test_Fake_Graph

def get_train_validate_test(dataset):
    realFileName = 'Datasets/' + dataset + '/realData.csv'
    fakeFileName = 'Datasets/' + dataset + '/fakeData.csv'
    train_Real_Graph, train_Fake_Graph, test_Real_Graph, test_Fake_Graph = structuralGraph(realFileName, fakeFileName, dataset)
    node2vecReFile = "Datasets/node2vecFeature/" + dataset + "Feature.txt"
    data = pd.read_csv(node2vecReFile, sep=' ', skiprows=1, header=None)
    edges = np.array(data.iloc[:, 0:1]) + np.array(data.iloc[:, 1:2])
    embeddings = np.array(data.iloc[:, 2:66])
    nodeL = np.array(data.iloc[:, 0:1])
    nodeR = np.array(data.iloc[:, 1:2])
    train_data = []
    test = []
    for i in range(len(edges)):
        edgeFeature = edgeFeatures(" ")
        nodel = int(re.sub("\D", "", nodeL[i][0]))
        noder = int(re.sub("\D", "", nodeR[i][0]))
        if train_Real_Graph.has_edge(nodel, noder) or train_Fake_Graph.has_edge(nodel, noder): 
            if train_Real_Graph.has_edge(nodel, noder):
                label = 1
                type = train_Real_Graph.get_edge_data(nodel, noder)['relationship']
            else:
                label = 0
                type = train_Fake_Graph.get_edge_data(nodel, noder)['relationship']
            edgeFeature.embeddings = embeddings[i]
            edgeFeature.label = label
            edgeFeature.type = type
            train_data.append(edgeFeature)
        elif test_Real_Graph.has_edge(nodel, noder) or test_Fake_Graph.has_edge(nodel, noder):  
            if test_Real_Graph.has_edge(nodel, noder):
                label = 1
                type = test_Real_Graph.get_edge_data(nodel, noder)['relationship']
            else:
                label = 0
                type = test_Fake_Graph.get_edge_data(nodel, noder)['relationship']
            edgeFeature.embeddings = embeddings[i]
            edgeFeature.label = label
            edgeFeature.type = type
            test.append(edgeFeature)
        else:
            continue

    train, validate = train_test_split(train_data, test_size=0.2)  
    train_dataset = []
    validate_dataset = []
    test_dataset = []
    for index, element in enumerate(train):
        vectors = torch.tensor(element.embeddings, dtype=torch.float32)
        label = torch.tensor(element.label, dtype=torch.float32)
        type = torch.tensor(element.type, dtype=torch.float32)
        m = [vectors, label, type]
        train_dataset.append(m)
    for index, element in enumerate(validate):
        vectors = torch.tensor(element.embeddings, dtype=torch.float32)
        label = torch.tensor(element.label, dtype=torch.float32)
        type = torch.tensor(element.type, dtype=torch.float32)
        m = [vectors, label, type]
        validate_dataset.append(m)
    for index, element in enumerate(test):
        vectors = torch.tensor(element.embeddings, dtype=torch.float32)
        label = torch.tensor(element.label, dtype=torch.float32)
        type = torch.tensor(element.type, dtype=torch.float32)
        m = [vectors, label, type]
        test_dataset.append(m)
    print('train length', len(train_dataset))
    print('validate length', len(validate_dataset))
    print('test length', len(test_dataset))

    # Define DataLoader globally
    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
    validate_loader = DataLoader(dataset=validate_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, validate_loader, test_loader

# Function to convert torch tensors to variable
def to_var(x):
    if torch.cuda.is_available():
        x = x.cuda()
    return Variable(x)

# Redesigned Model with Genetic Algorithm Integration
class adversarial_neural_networks(nn.Module):
    def __init__(self, predicted_Type, hidden_dim):
        super(adversarial_neural_networks, self).__init__()
        self.predicted_Type = predicted_Type
        self.hidden_dim = hidden_dim

        self.predictor = nn.Sequential()
        self.predictor.add_module('exta_Conv1', nn.Conv1d(in_channels=1, out_channels=1, kernel_size=10, stride=1, padding=0))
        self.predictor.add_module('fully_connected_layer1', nn.Linear(55, self.hidden_dim))

        self.predictor_classifier = nn.Sequential()
        self.predictor_classifier.add_module('c_fc1', nn.Linear(self.hidden_dim, 24))
        self.predictor_classifier.add_module('c_fc1_relu', nn.ReLU())
        self.predictor_classifier.add_module('c_fc2', nn.Linear(24, 16))
        self.predictor_classifier.add_module('c_fc2_relu', nn.ReLU())
        self.predictor_classifier.add_module('c_fc3', nn.Linear(16, 2))
        self.predictor_classifier.add_module('c_softmax', nn.Softmax(dim=1))

        self.discriminative_classifier = nn.Sequential()
        self.discriminative_classifier.add_module('d_fc1', nn.Linear(self.hidden_dim, 16))
        self.discriminative_classifier.add_module('relu_f1', nn.ReLU())
        self.discriminative_classifier.add_module('d_fc2', nn.Linear(16, self.predicted_Type))
        self.discriminative_classifier.add_module('d_softmax', nn.Softmax(dim=1))

    def forward(self, embeddings):
        embeddings = self.predictor(embeddings)
        shared_embeddings = embeddings.view_as(embeddings)
        result = self.predictor_classifier(shared_embeddings)
        return result, shared_embeddings

# Fitness Function for GA
def fitness_function(ga_instance, solution, solution_idx):
    global model, train_loader
    model = adversarial_neural_networks(2, hidden_dim)
    model_params = solution
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Train model for a few epochs
    for epoch in range(10):  # Limited epochs for GA fitness check
        for inputs, labels, _ in train_loader:
            inputs, labels = to_var(inputs), to_var(labels)
            optimizer.zero_grad()
            outputs, _ = model(inputs)
            loss = nn.CrossEntropyLoss()(outputs, labels.long())
            loss.backward()
            optimizer.step()

    # Fitness value is the model's accuracy on the validation set
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels, _ in validate_loader:
            inputs, labels = to_var(inputs), to_var(labels)
            outputs, _ = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Return fitness as the accuracy
    return correct / total

# PyGAD setup
ga = pygad.GA(num_generations=num_generations,
              num_parents_mating=4,
              fitness_func=fitness_function,
              sol_per_pop=population_size,
              num_genes=2,  # 1 gene for learning rate, 1 for hidden dimension
              gene_type=float,
              gene_space=[(0.0001, 0.1), (4, 64)],  # Learning rate, hidden dim
              crossover_type=crossover_type,
              crossover_probability=crossover_probability,
              mutation_type="random",
              mutation_probability=mutation_probability)

ga.run()

# Optimal solution found
solution, solution_fitness = ga.best_solution()
print("Best Solution:", solution)
print("Best Fitness Value:", solution_fitness)


name 'train_loader' is not defined
Traceback (most recent call last):
  File "C:\Users\uclab1\AppData\Local\Programs\Python\Python311\Lib\site-packages\pygad\pygad.py", line 1708, in cal_pop_fitness
    fitness = self.fitness_func(self, sol, sol_idx)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\uclab1\AppData\Local\Temp\ipykernel_3772\4036893793.py", line 195, in fitness_function
    for inputs, labels, _ in train_loader:
                             ^^^^^^^^^^^^
NameError: name 'train_loader' is not defined
name 'train_loader' is not defined
Traceback (most recent call last):
  File "C:\Users\uclab1\AppData\Local\Programs\Python\Python311\Lib\site-packages\pygad\pygad.py", line 1914, in run
    self.last_generation_fitness = self.cal_pop_fitness()
                                   ^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\uclab1\AppData\Local\Programs\Python\Python311\Lib\site-packages\pygad\pygad.py", line 1871, in cal_pop_fitness
    raise ex
  File "C:\Users\

NameError: name 'train_loader' is not defined